In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as scs
import os
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import norm, kurtosis
from scipy.stats import skew
import scipy.stats
import pickle
import statistics as stats
import statsmodels as sm

In [ ]:
data = pd.read_excel(
     os.path.join("realisedvariancesourceredclosingprice.xlsx"),
     engine='openpyxl',
)
list_dates_formated = []
list_dates = list(data['Unnamed: 0'])
list_names = list(data['Symbol'])
list_closing_prices= list(data['close_price'])
A_all = []
size_current = len(list_dates)
for i in range(0,size_current):
    date = list_dates[i].split(" ")[0].split("-")
    year = int(date[0])
    month = int(date[1])
    day = int(date[2])    
    list_dates_formated.append(dt.datetime(year,month,day))
    A_all.append([dt.datetime(year,month,day),list_names[i],list_closing_prices[i]])
df_all_returns = pd.DataFrame(A_all)
df_all_returns.columns = ["Date","Names","close_price"]      

def returns_real_data(index,start_date,end_date):
    Prices = df_all_returns.iloc[list(df_all_returns['Names'] == list_names_unique[index])]
    True_False = []
    Dates_between = list(Prices['Date'] )
    for date in Dates_between:
        if start_date <= date <= end_date:
            True_False.append(True)
        else: 
            True_False.append(False)
    #take the range of date we are interested in 
    Prices_df = Prices.iloc[True_False]
    Prices_df.dropna(axis=1)
    Prices_list = list(Prices_df['close_price'])
    Returns_list = []
    for i in range(len(Prices_list)-1):
        Returns_list += [np.log(Prices_list[i+1]/Prices_list[i])]
    return(Returns_list)

list_names_unique = sorted(list(set(list(df_all_returns['Names']))))



In [ ]:
plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.size'] = 20
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.titlesize'] = 24
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['font.family'] = 'serif'

In [ ]:
plt.style.use('seaborn')
plt.rcParams['figure.dpi'] = 300

In this file, the example is provided for the rough fSV model but the same has been done for the other models

In [ ]:
with open("SF_r.txt", "rb") as fp:   # Unpickling
    log_returns = pickle.load(fp)

In [ ]:
#log_returns = returns_real_data(0,dt.datetime(2000,1,1),dt.datetime(2000,6,1)) line for the real data

In [ ]:
skew(log_returns)

In [ ]:
kurtosis(log_returns)

In [ ]:
r = stats.mean(log_returns)
M =1
sigma =np.sqrt(stats.variance(log_returns))

In [ ]:
len(log_returns)

In [ ]:
plt.figure(figsize=(2,4))

In [ ]:
plt.hist(log_returns, bins=70, label='frequency', density = True)
plt.grid(True)
plt.xlabel('log-return')
plt.ylabel('frequency')
plt.title("A histogram of the daily returns on S&P 500 index with a fitted normal distribution")
x = np.linspace(plt.axis()[0], plt.axis()[1])
plt.plot(x, scs.norm.pdf(x, loc=r / M, scale=sigma / np.sqrt(M)),'r', lw=2.0, label='pdf')
plt.legend()

In [ ]:
standardised_log_returns= []
for i in range(len(log_returns)):
    standardised_log_returns += [(log_returns[i]-r)/sigma]

In [ ]:
sm.graphics.gofplots.qqplot(np.array(standardised_log_returns), line='s')
plt.grid(True)
plt.title("A normal QQ-plot of standardised daily returns for S&P 500 ")
plt.xlabel('theoretical quantiles')
plt.ylabel('sample quantiles')

In [ ]:
abs_log_returns = []
for i in range(len(log_returns)):
    abs_log_returns += [abs(log_returns[i])]

In [ ]:
plt.figure()
fig, (ax1, ax2) = plt.subplots(1,2)
fig.set_size_inches(15, 4)
ax1.acorr(np.array(log_returns),maxlags=15)
ax1.set(xlabel='Lags', ylabel='',title="Empirical autocorrelation plot of the daily returns of the S&P 500")
ax2.acorr(np.array(abs_log_returns),maxlags=15)
ax2.set(xlabel='Lags', ylabel='',title="Empirical autocorrelation plot of the daily absolute returns of the S&P 500")